In [1]:
import pymssql 

conn = pymssql.connect(
    server='mipt.database.windows.net',
    user='student03',
    password='Student_03',
    database='demo',
)


from tabulate import tabulate

def exec(query: str):
    cursor = conn.cursor()  
    cursor.execute(query) 
    
def q(query: str, *args):
    cursor = conn.cursor()  
    cursor.execute(query) 
    row = cursor.fetchone()  
    rows = []
    while row:  
        rows.append(row)
        row = cursor.fetchone()  
    print(tabulate(rows, headers=args))

In [2]:
q("""
    SELECT name, database_id, create_date  
    FROM sys.databases;  
""")

------  -  --------------------------
master  1  2021-11-08 06:18:56.827000
demo    5  2021-11-08 15:01:07.873000
------  -  --------------------------


## Task 1

**1 -**

**2**

**3**

4

5

6

7

8\. no

9\. no

10\. no

11\. no

12\. no

13\. no

14\. no

15\. no

16\. no

17\. -

18. Сколько в среднем обслуживает клиентов менеджер филиала.

19. Сколько всего клиентов обслужил филиал за определенный период.

In [5]:
q("""--sql
    select count(sales.checkId), branch.branchName
    from distributor.sales
    inner join distributor.branch on (sales.branchId = branch.branchId)
    group by sales.branchId, branch.branchName
""", "Clients served", "Branch")

  Clients served  Branch
----------------  ---------------
          360500  Екатеринбург
            1063  ИнтернетМагазин
          773092  Москва
          424876  Новосибирск
            6124  Пятигорск
          351504  Самара


20. Какой менеджер обслужил в филиале, максимальное кол-во клиентов

In [6]:
q("""--sql
    with temp(managerId, checksSold) as (
        select salesManagerId, count(sales.checkId)
        from distributor.sales
        where salesManagerId is not Null
        group by salesManagerId
    )

    select managerId, surname, [names], checksSold
    from temp
    inner join distributor.salesManager on (salesManager.salesManagerId = temp.managerId)
    where checksSold = (
        select max(checksSold)
        from temp
    )
""", "id", "surname", "name", "clients")

  id  surname    name      clients
----  ---------  ------  ---------
  82  Фирсова    Елена       78087


21. Какой менеджер, принес максимальную выручку в филиале за определенный месяц

In [57]:
q("""--sql
    select max(salesRub) as maxSalesRub, salesManagerId
    from distributor.sales 
    where
        branchId = 4 and
        month(dateId) = 8
    group by salesManagerId
""", "sales in rub", "Manager ID")

  sales in rub    Manager ID
--------------  ------------
      185100
       39019.8            53
      218825              87
      195663              88
      343230              95
      228000              96
      122248             100
      140315             131
      202600             141
       36000             152
       40826.5           165
      146192             197
      138942             198
       65312.4           204
       36127.6           234
       75093.4           238
      411211             240


22. Рассчитать средний чек клиенту по выбранному менеджеру

In [59]:
q("""--sql
    select top(10) avg(salesRub), salesManager.surname
    from distributor.sales
    inner join distributor.salesManager on (salesManager.salesManagerId = sales.salesManagerId)
    group by sales.salesManagerId, salesManager.surname
""", "Average check", "Sales manager surname")

  Average check  Sales manager surname
---------------  -----------------------
        1926.75  Аброскина
        1094.02  Андреева
        3219.91  Антонов
       20129.1   Бакуров
       19984.7   Баранов
        2612.61  Баранов
        4378.14  Березина
        7175.51  БОТВИЧ
        9753.49  Валов
        3729.43  Васильев


23. Рассчитать средний чек клиента по филиалу

In [60]:
q("""--sql
    select avg(salesRub), branchName
    from distributor.sales
    inner join distributor.branch on (branch.branchId = sales.branchId)
    group by branch.branchId, branchName
""", "Avarage check in rubles", "Branch")

  Avarage check in rubles  Branch
-------------------------  ---------------
                  4584.42  Екатеринбург
                  2142.03  ИнтернетМагазин
                  7309.87  Москва
                  5978.9   Новосибирск
                 14346.4   Пятигорск
                  4976.34  Самара


24. Средний чек клиента по менеджерам внутри филиалов

In [10]:
# TODO 

25

In [11]:
# TODO 

26

In [12]:
# TODO 

27. Рассчитать АВС товарных позиций

In [13]:
# TODO 

28\. -

29\. -

30\. -

31\. -

32. Есть менеджеры, которые продают в более чем одном филиале, найти их!

In [12]:
q("""--sql
    with sms(managerId, numberOfBranches)
    as
    (
        select sm.salesManagerId, count(distinct branch.branchId)
        from distributor.sales
        inner join distributor.branch on (sales.branchId = branch.branchId)
        inner join distributor.salesManager as sm on (sales.salesManagerId = sm.salesManagerId)
        group by sm.salesManagerId
        having count(distinct branch.branchId) > 1
    )

    select managerId, numberOfBranches, salesManager.surname, salesManager.[names]
    from sms
    inner join distributor.salesManager on (sms.managerId = salesManager.salesManagerId)
""", "ID", "nBranches", "Surname", "Name")

  ID    nBranches  Surname      Name
----  -----------  -----------  ---------------
   7            2  Березина     Стелла
  10            2  Васильев     Дмитрий
  13            2  Воробьева    Юлия
   9            2  Валов        Игорь
  19            2  Долганов     Геннадий
  31            2  Карпухина    Елена Андреевна
  28            2  Исакова      Евгения
  20            2  Дудкина      Татьяна
  74            2  Семенов      Олег
  37            2  Комарова     Василина
  53            2  Меньшиков    Сергей
  47            2  Мальшин      Олег
  67            3  Рензяк       Диана
  76            2  Собакин      Денис
  78            3  Ставицкая    Анастасия
  85            2  Чуканова     Инна
  87            2  Шашкина      Полина
  89            2  Ширяев       Александр
  83            2  Хакунов      Анзор
  34            2  Ковалевская  Алёна
  36            2  Комаров      Эдуард
  54            2  Минеева      Ксения


33. Найти сумму продаж по менеджерам, со следующими условиями. Какими условиями?

34. Выгрузки с датами, различными. The fuck?

35. Показать, что среднее кол-во чеков и среднее по транзакции это разные вещи.

In [15]:
# todo

36\. Найти количество уникальных счетов при условиях. Каких условиях?

## Task 2

In [ ]:
Рассчитать выручку компании в разрезе: Год – Месяц – Выручка компании. Представленные данные отсортировать: Год, Месяц

In [17]:
q("""--sql
    select top(10) sum(distributor.singleSales.salesRub)
    from distributor.singleSales
    group by
        distributor.singleSales.companyName, 
        YEAR(distributor.singleSales.dateId),
        month(distributor.singleSales.dateId)
""")

--------------
   9.95263e+07
   1.688e+08
   1.32953e+08
   1.57553e+08
   1.8168e+08
   1.47678e+08
   1.70409e+08
   1.92769e+08
   1.93963e+08
4933.8
--------------
